In [48]:
import keras
import tensorflow as tf
import numpy as np

In [278]:
model = keras.Sequential([
    keras.layers.ConvLSTM2D(10, (32,64), strides=(32,32), input_shape=(32, 32, 8192, 1), padding='valid', activation='tanh', recurrent_activation='hard_sigmoid',activity_regularizer=None, return_sequences=False, dropout=0.0, recurrent_dropout=0.0),
    keras.layers.Flatten(),
    keras.layers.Dense(units=2, activation='linear'),
])

lr = 0.001
optimizer = keras.optimizers.Adam(lr)
loss='categorical_crossentropy'
model.compile(optimizer,loss, metrics=['accuracy'])

In [279]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_158 (ConvLSTM2D (None, 1, 255, 10)        901160    
_________________________________________________________________
flatten_19 (Flatten)         (None, 2550)              0         
_________________________________________________________________
dense_29 (Dense)             (None, 2)                 5102      
Total params: 906,262
Trainable params: 906,262
Non-trainable params: 0
_________________________________________________________________


In [366]:
from keras.layers import Input, ConvLSTM2D, RepeatVector,UpSampling2D,Reshape,Conv3D,Lambda,Flatten,Dense
from keras.models import Model
import keras.backend as k

def stacklayer(decoded):
    x = [decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded,decoded]
    return k.stack(x,axis=1)

inputs = Input(shape=(32, 32, 8192, 1))
encoded = ConvLSTM2D(10, (32,32), strides=(32,32), padding='valid', activation='tanh', recurrent_activation='hard_sigmoid',activity_regularizer=None, return_sequences=False, dropout=0.0, recurrent_dropout=0.0)(inputs)
encoded = Flatten()(encoded)
encoded = Dense(2,activation = 'linear')(encoded)


decoded = Dense(2560,activation = 'linear')(encoded)
decoded = Reshape((-1,256,10))(decoded)
decoded = UpSampling2D((32,32))(decoded)
decoded = Lambda(stacklayer)(decoded)
decoded = ConvLSTM2D(10, (32,32), strides=(1,1), padding='same', activation='tanh', recurrent_activation='hard_sigmoid',activity_regularizer=None, return_sequences=True, dropout=0.0, recurrent_dropout=0.0)(decoded)
decoded = Conv3D(1,(32,32,1), strides=(1,1,1), padding='same',activation = 'sigmoid')(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)
sequence_autoencoder.compile(optimizer='adam', loss='mse')

In [367]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_114 (InputLayer)       (None, 32, 32, 8192, 1)   0         
_________________________________________________________________
conv_lst_m2d_213 (ConvLSTM2D (None, 1, 256, 10)        450600    
_________________________________________________________________
flatten_27 (Flatten)         (None, 2560)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 2)                 5122      
Total params: 455,722
Trainable params: 455,722
Non-trainable params: 0
_________________________________________________________________


In [368]:
sequence_autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_114 (InputLayer)       (None, 32, 32, 8192, 1)   0         
_________________________________________________________________
conv_lst_m2d_213 (ConvLSTM2D (None, 1, 256, 10)        450600    
_________________________________________________________________
flatten_27 (Flatten)         (None, 2560)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 2)                 5122      
_________________________________________________________________
dense_43 (Dense)             (None, 2560)              7680      
_________________________________________________________________
reshape_58 (Reshape)         (None, 1, 256, 10)        0         
_________________________________________________________________
up_sampling2d_86 (UpSampling (None, 32, 8192, 10)      0         
__________